In [1]:
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, UpSampling2D, Flatten, Dense

def load_dataset(csv_file):
    df = pd.read_csv(csv_file)

    # Read image paths and enhanced images from the dataset
    image_paths = df['Original Path'].values
    camera_model = df['Camera Model'].values
    focal_length = df['Focal Length'].values
    f_stop = df['F-stop'].values
    exposure_time = df['Exposure Time'].values
    iso = df['ISO'].values
    exposure_bias = df['Exposure Bias'].values
    flash_status = df['Flash Status'].values
    rating = df['Rating'].values
    enhanced_images = df['Enhanced Image'].values

    # Load and preprocess images
    images = []
    for path in image_paths:
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        image = cv2.resize(image, (250, 250))  # Resize image to 250x250 pixels
        image = image / 255.0  # Normalize pixel values to the range of [0, 1]
        images.append(image)

    images = np.array(images)

    # Map flash status to binary values
    flash_status = np.array([1 if status == 'on' else 0 for status in flash_status])

    return images, camera_model, focal_length, f_stop, exposure_time, iso, exposure_bias, flash_status, rating

# Replace 'dataset.csv' with the path to your CSV file
images, camera_model, focal_length, f_stop, exposure_time, iso, exposure_bias, flash_status, rating = load_dataset(r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\brightness\AIE DATASET.csv")

# Define the CNN model architecture
def create_model(input_shape, num_camera_models):
    model = Sequential()

    # Add convolutional layers
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'))

    # Flatten the camera attributes
    model.add(Flatten())

    # Add dense layers for camera attributes
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(num_camera_models, activation='softmax'))

    # Add final dense layers for rating and flash status
    model.add(Dense(32, activation='relu'))
    model.add(Dense(5, activation='softmax'))  # Output for rating
    model.add(Dense(1, activation='sigmoid'))  # Output for flash status

    # Reshape back to image dimensions
    model.add(Dense(np.prod(input_shape), activation='relu'))
    model.add(tf.keras.layers.Reshape(input_shape))

    # Add upsampling layers
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(Conv2D(3, kernel_size=(3, 3), activation='relu', padding='same'))

    return model

# Define the input shape based on your images
input_shape = (250, 250, 3)
num_camera_models = len(np.unique(camera_model))
model = create_model(input_shape, num_camera_models)

# Compile and train the model
model.compile(optimizer='adam', loss=['categorical_crossentropy', 'binary_crossentropy'], metrics=['accuracy'])

# Create a dictionary for input tensors
inputs = {
    'conv2d_1_input': images,  # Replace 'conv2d_1_input' with the actual input layer name
    'camera_model': camera_model,
    'focal_length': focal_length,
    'f_stop': f_stop,
    'exposure_time': exposure_time,
    'iso': iso,
    'exposure_bias': exposure_bias,
    'flash_status': flash_status,
    'rating': rating
}

# Replace 'inputs' with your actual input data
model.fit(inputs, [rating, flash_status], batch_size=32, epochs=10, validation_split=0.2)

#  Save the trained model
model.save('image_refinement_model.h5')

# Load the trained model and predict on a new image
def load_and_preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    image = cv2.resize(image, (250, 250))  # Resize image to 250x250 pixels
    image = image / 255.0  # Normalize pixel values to the range of [0, 1]
    return image

def predict_enhanced_image(model, image_path):
    input_image = load_and_preprocess_image(image_path)
    input_image = np.expand_dims(input_image, axis=0)  # Add batch dimension

    # Load the trained model
    loaded_model = tf.keras.models.load_model('image_refinement_model.h5')

    # Predict the enhanced image, rating, and flash status
    enhanced_image, rating_prediction, flash_status_prediction = loaded_model.predict(input_image)

    # Rescale the enhanced image to the range of [0, 255]
    enhanced_image = enhanced_image * 255.0
    enhanced_image = np.clip(enhanced_image, 0, 255)
    enhanced_image = enhanced_image.astype(np.uint8)

    return enhanced_image, rating_prediction, flash_status_prediction

# Replace 'input_image_path' with the path to your input image
enhanced_image, rating_prediction, flash_status_prediction = predict_enhanced_image(model, r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Raw\DSC_2478.JPG")


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\MAHESH\\Desktop\\SRFP INTERNSHIP\\train_in_1\\brightness\\AIE DATASET.csv'

In [ ]:
import cv2
import numpy as np

def normalize_adjustments(adjustments):
    # Normalize adjustments to a common range
    normalized_adjustments = adjustments / np.max(adjustments)
    return normalized_adjustments

def calculate_optimal_gamma(image):
    # Calculate optimal gamma value using an optimization technique
    # Here, we'll use a simple brute-force search
    best_gamma = 1.0
    best_score = -1.0
    
    for gamma in np.arange(0.1, 5.1, 0.1):
        adjusted_image = adjust_gamma(image, gamma)
        score = calculate_image_quality(adjusted_image)
        
        if score > best_score:
            best_score = score
            best_gamma = gamma
    
    return best_gamma

def adjust_gamma(image, gamma):
    # Apply gamma correction to the image
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in np.arange(0, 256)]).astype(np.uint8)
    adjusted_image = cv2.LUT(image, table)
    return adjusted_image

def adjust_contrast(image, gamma):
    # Apply contrast adjustment using gamma correction
    adjusted_image = adjust_gamma(image, gamma)
    return adjusted_image

def adjust_brightness(image, brightness):
    # Apply brightness adjustment
    adjusted_image = cv2.convertScaleAbs(image, alpha=1.0, beta=brightness)
    return adjusted_image

def adjust_tint(image, tint):
    # Apply tint adjustment
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv_image)
    h_adjusted = np.clip(h + tint, 0, 255)
    adjusted_hsv = cv2.merge([h_adjusted, s, v])
    adjusted_image = cv2.cvtColor(adjusted_hsv, cv2.COLOR_HSV2BGR)
    return adjusted_image

def adjust_saturation(image, saturation):
    # Apply saturation adjustment
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv_image)
    s_adjusted = np.clip(s + saturation, 0, 255)
    adjusted_hsv = cv2.merge([h, s_adjusted, v])
    adjusted_image = cv2.cvtColor(adjusted_hsv, cv2.COLOR_HSV2BGR)
    return adjusted_image

# Example usage
original_image = cv2.imread(r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Raw\DSC_2478.JPG")
enhanced_image = cv2.imread(r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Done\DSC_2478.jpg")

# Assuming adjustments are provided as arrays or values
initial_adjustments = np.array([0.8, 20, -10, 30])  # [contrast, brightness, tint, saturation]

# Normalize adjustments
normalized_adjustments = normalize_adjustments(initial_adjustments)

# Calculate optimal gamma
optimal_gamma = calculate_optimal_gamma(original_image)

# Apply adjustments to the original image
adjusted_image = original_image.copy()
adjusted_image = adjust_contrast(adjusted_image, optimal_gamma)
adjusted_image = adjust_brightness(adjusted_image, normalized_adjustments[1])
adjusted_image = adjust_tint(adjusted_image, normalized_adjustments[2])
adjusted_image = adjust_saturation(adjusted_image, normalized_adjustments[3])

# Evaluate stability or save the adjusted image
cv2.imwrite(r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Done\DSC_2478dont.jpg", adjusted_image)